# Find KEGG associations

This notebokk will create a table that has the KEGG pathways that are associated with the most stable and least stable core genes.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import random
import pandas as pd
from scripts import paths, utils, modules, annotations

random.seed(1)

In [2]:
# Output files
pao1_core_stable_similarity_filename = "pao1_core_stable_associations.tsv"
pa14_core_stable_similarity_filename = "pa14_core_stable_associations.tsv"

In [3]:
# Load transcriptional similarity df
pao1_similarity_scores_filename = "pao1_similarity_scores.tsv"
pa14_similarity_scores_filename = "pa14_similarity_scores.tsv"

pao1_similarity_scores = pd.read_csv(
    pao1_similarity_scores_filename, sep="\t", header=0, index_col=0
)
pa14_similarity_scores = pd.read_csv(
    pa14_similarity_scores_filename, sep="\t", header=0, index_col=0
)

In [4]:
pao1_similarity_scores.head()

,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label
PAO1 id,,,,,
PA1642,PA14_43280,0.393037,4.089216e-197,selD,NaN
PA0481,PA14_06280,0.242300,2.526173e-72,NaN,NaN
PA3714,PA14_16350,0.335471,7.698982e-141,NaN,NaN
PA1164,PA14_49350,0.389940,8.702679e-194,NaN,NaN
PA0749,PA14_54600,0.336135,2.004630e-141,NaN,NaN


In [5]:
# Load KEGG pathway data
pao1_pathway_filename = "https://raw.githubusercontent.com/greenelab/adage/7a4eda39d360b224268921dc1f2c14b32788ab16/Node_interpretation/pseudomonas_KEGG_terms.txt"

In [6]:
pao1_pathways = annotations.load_format_KEGG(pao1_pathway_filename)
pao1_pathways.head()

,1,2
0,,
KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,10,"{PA2011, PA2003, PA2001, PA4785, PA2553, PA392..."
KEGG-Pathway-pae00071: Fatty acid degradation,32,"{PA1748, PA2001, PA1535, PA1736, PA0879, PA118..."
KEGG-Pathway-pae00903: Limonene and pinene degradation,9,"{PA1748, PA2475, PA3331, PA1821, PA3426, PA102..."
KEGG-Pathway-pae00380: Tryptophan metabolism,27,"{PA1748, PA2001, PA0704, PA1736, PA4163, PA358..."
KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,16,"{PA4669, PA2001, PA4044, PA4785, PA3803, PA255..."


## Pathway annotations to PA14

The annotations we have are only for PAO1 genes, so we will map PAO1 core genes to PA14 core genes to add annotations to PA14. This is possible since we are focused on only core genes, which have homologs between PAO1 and PA14

In [7]:
pao1_annotation_filename = paths.GENE_PAO1_ANNOT
gene_mapping_pao1 = utils.get_pao1_pa14_gene_map(pao1_annotation_filename, "pao1")

In [8]:
gene_mapping_pao1 = gene_mapping_pao1["PA14_ID"].to_frame()

## Get pathway associations for most and least stable genes

In [9]:
# Get most and least stable core genes
most_stable_genes = pao1_similarity_scores[
    pao1_similarity_scores["label"] == "most stable"
].index
least_stable_genes = pao1_similarity_scores[
    pao1_similarity_scores["label"] == "least stable"
].index

In [10]:
def get_associated_pathways(genes_):
    rows = []
    for gene_id in genes_:
        pathway_bool = [
            gene_id in pao1_pathways.loc[pathway, 2] for pathway in pao1_pathways.index
        ]
        found_pathways = list(pao1_pathways[pathway_bool].index)
        rows.append({"gene id": gene_id, "pathways present": found_pathways})
    return pd.DataFrame(rows).set_index("gene id")

In [11]:
most_stable_associations = get_associated_pathways(most_stable_genes)
most_stable_associations.head()

,pathways present
gene id,
PA0355,[]
PA3655,[]
PA0620,[]
PA0956,[KEGG-Pathway-pae00970: Aminoacyl-tRNA biosynt...
PA3834,[KEGG-Pathway-pae00970: Aminoacyl-tRNA biosynt...


In [12]:
least_stable_associations = get_associated_pathways(least_stable_genes)
least_stable_associations.head()

,pathways present
gene id,
PA3577,[]
PA2018,[KEGG-Pathway-pae00312: beta-Lactam resistance]
PA1195,[]
PA3206,[]
PA0605,[KEGG-Module-M00193: Putative spermidine/putre...


In [13]:
# Concatenate dataframes
pao1_all_associations = pd.concat([most_stable_associations, least_stable_associations])
pao1_all_associations.head()

# TO DO: Rename index col

,pathways present
gene id,
PA0355,[]
PA3655,[]
PA0620,[]
PA0956,[KEGG-Pathway-pae00970: Aminoacyl-tRNA biosynt...
PA3834,[KEGG-Pathway-pae00970: Aminoacyl-tRNA biosynt...


In [14]:
# Map PA14 gene ids
pa14_all_associations = pao1_all_associations.merge(
    gene_mapping_pao1, left_index=True, right_index=True
)
pa14_all_associations.set_index("PA14_ID", inplace=True)
pa14_all_associations.head()

,pathways present
PA14_ID,
PA14_04650,[]
PA14_17070,[]
PA14_08050,[]
PA14_51900,[KEGG-Pathway-pae00970: Aminoacyl-tRNA biosynt...
PA14_14440,[KEGG-Pathway-pae00970: Aminoacyl-tRNA biosynt...


In [15]:
# Merge KEGG associations with transcriptional similarity information
pao1_all_associations = pao1_all_associations.merge(
    pao1_similarity_scores, left_index=True, right_index=True, how="left"
)
pa14_all_associations = pa14_all_associations.merge(
    pa14_similarity_scores, left_index=True, right_index=True, how="left"
)

In [16]:
pao1_all_associations.head()

,pathways present,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label
gene id,,,,,,
PA0355,[],PA14_04650,0.508206,0.0,pfpI,most stable
PA3655,[],PA14_17070,0.579558,0.0,tsf,most stable
PA0620,[],PA14_08050,0.545196,0.0,NaN,most stable
PA0956,[KEGG-Pathway-pae00970: Aminoacyl-tRNA biosynt...,PA14_51900,0.581097,0.0,proS,most stable
PA3834,[KEGG-Pathway-pae00970: Aminoacyl-tRNA biosynt...,PA14_14440,0.505727,0.0,valS,most stable


In [17]:
# Save
pao1_all_associations.to_csv(pao1_core_stable_similarity_filename, sep="\t")
pa14_all_associations.to_csv(pa14_core_stable_similarity_filename, sep="\t")

**Takeaway:**

Based on the pathways associated with the most and least stable core genes, we find that the most stable core genes tend to be associated with pathways related to cellular maintenance including: protein transport systems, ribosomes, metabolism, type III, IV secretion system which mediates virulence.

There are far fewer KEGG pathways that least stable core genes are found to be associated with. The least stable core genes are mostly associated with different types of metabolism.

A google doc containing the most and least stable core genes and some information about them is [here](https://docs.google.com/spreadsheets/d/1SqEyBvutfbsOTo4afg9GiEzP32ZKplkN1a6MpAQBvZI/edit?usp=sharing).